**Import Library**

In [1]:
from time import sleep
from selenium import webdriver
import pandas as pd

**Choose the Data Date Periode**

In [2]:
#Open The Website Page
driver = webdriver.Chrome('chromedriver')
driver.get('https://www.bi.go.id/en/statistik/informasi-kurs/jisdor/Default.aspx')
sleep(1)

#Input The Particular First Date (t2021-01-01)
driver.find_element_by_xpath('//*[@id="TextBoxFrom"]').click()
sleep(1)
driver.find_element_by_xpath('/html/body/div[2]/div[1]/table/thead/tr[2]/th[2]').click()
sleep(1)
driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr/td/span[1]').click()
sleep(1)
driver.find_element_by_xpath('/html/body/div[2]/div[1]/table/tbody/tr[1]/td[6]').click()
sleep(2)

#Input The Particular Last Date (2021-08-18)
driver.find_element_by_xpath('//*[@id="TextBoxDateTo"]').click()
sleep(1)
driver.find_element_by_xpath('/html/body/div[2]/div[1]/table/thead/tr[2]/th[2]').click()
sleep(1)
driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr/td/span[8]').click()
sleep(1)
driver.find_element_by_xpath('/html/body/div[2]/div[1]/table/tbody/tr[4]/td[4]').click()
sleep(1)

#Click the Button "Search" to Search The Data
driver.find_element_by_xpath('//*[@id="ctl00_ctl44_g_635f8117_c448_4d65_b807_5df7064c3fe7_ctl00_ButtonCari"]').click()

**Get the Total of Data Website Page**

In [3]:
#Loop the "Next" button to get the last page from the website
page = []
i = 1
while i > 0:
    page.append(driver.find_element_by_class_name('page-link--custom.active').text)
    #Get the condition when the total last table rows equal 10
    if len(driver.find_elements_by_xpath('//*[@id="tableData"]/table/tbody/tr')) == 10:
        element = driver.find_element_by_xpath('//*[@id="ctl00_ctl44_g_635f8117_c448_4d65_b807_5df7064c3fe7_ctl00_DataPagerData"]/input[2]')
        driver.execute_script("arguments[0].click();", element)
        if driver.find_element_by_class_name('page-link--custom.active').text in page:
            break
        else:
            continue
    #Get the condition when the total last table rows not equal 10
    else:
        break

#Get the last website page content
max_page = page[-1]

**Conduct the reverse process to the first website page**

In [4]:
while driver.find_element_by_xpath('//*[@id="ctl00_ctl44_g_635f8117_c448_4d65_b807_5df7064c3fe7_ctl00_DataPagerData"]/span').text != '1':
    element = driver.find_element_by_xpath('//*[@id="ctl00_ctl44_g_635f8117_c448_4d65_b807_5df7064c3fe7_ctl00_DataPagerData"]/input[1]')
    driver.execute_script("arguments[0].click();", element)

**Get the table data in every page**

In [5]:
column1 = []
column2 = []
i = 1
while i > 0:
    columns = 2
    rows = len(driver.find_elements_by_xpath('//*[@id="tableData"]/table/tbody/tr'))
    for r in range(1,rows+1): #row
        for c in range(1,columns+1): #column
            if c%2 != 0:
                column1.append(driver.find_element_by_xpath('//*[@id="tableData"]/table/tbody/tr['+str(r)+']/td['+str(c)+']').text)
            else:
                column2.append(driver.find_element_by_xpath('//*[@id="tableData"]/table/tbody/tr['+str(r)+']/td['+str(c)+']').text)
    element = driver.find_element_by_xpath('//*[@id="ctl00_ctl44_g_635f8117_c448_4d65_b807_5df7064c3fe7_ctl00_DataPagerData"]/input[2]')
    driver.execute_script("arguments[0].click();", element)
    i+=1
    #Break the looping of website page condition
    if i > int(max_page):
        break

**Concatenate the tables from every website page**

In [6]:
column1 = pd.DataFrame(column1,columns = ['Date'])
column2 = pd.DataFrame(column2,columns = ['Rates'])
Data_BI = pd.concat([column1,column2],axis=1)

**Show The Concatenated Data**

In [7]:
Data_BI

,Date,Rates
0,18 Agustus 2021,"Rp14.384,00"
1,16 Agustus 2021,"Rp14.383,00"
2,13 Agustus 2021,"Rp14.388,00"
3,12 Agustus 2021,"Rp14.389,00"
4,10 Agustus 2021,"Rp14.397,00"
...,...,...
147,8 Januari 2021,"Rp14.058,00"
148,7 Januari 2021,"Rp13.938,00"
149,6 Januari 2021,"Rp13.926,00"
150,5 Januari 2021,"Rp13.945,00"


**Export The Data in CSV File Format**

In [8]:
Data_BI.to_csv('Homework Web Scraping.csv',index=False)